# Image-to-Composition Tracing

Trace real-world photographs into stylized stroke compositions using computer vision.

**Pipeline:** Image → Edge Detection (OpenCV Canny) → SVG Tracing (vtracer) → Stroke Extraction → Composition

**Key advantages over LLM-only generation:**
- Spatially precise — deterministic edge detection, no hallucinated coordinates
- Free — runs entirely locally, no API costs
- Fast — full pipeline under 1 second per image
- Parameter tunable — adjust Canny thresholds and simplification in real-time

**Optional:** Use vision LLM (Ollama or Claude) to auto-detect subject and generate tags.

In [ ]:
# Cell 1: Setup — imports and dependency checks
%matplotlib inline
import time
import matplotlib.pyplot as plt

from helpers import (
    Composition, validate, bounding_box, count_strokes, count_points,
    draw, draw_grid, draw_comparison,
    get_curated, save_compositions,
    load_image, download_image, search_images, show_image, show_image_grid, show_side_by_side,
    detect_edges, trace_to_svg, svg_to_strokes, trace_image, trace_with_params,
)
from helpers.validate import score_breakdown
from helpers.vision import label_with_ollama, label_with_claude, check_vision_model

# Verify core dependencies
import cv2
import vtracer
import svgpathtools
from PIL import Image
print(f"OpenCV: {cv2.__version__}")
print(f"vtracer: OK")
print(f"svgpathtools: OK")
print(f"Pillow: {Image.__version__}")

# Check optional vision model
print(f"\n{check_vision_model()}")

In [ ]:
# Cell 2: Load an image
# Option A: From URL
img = load_image("https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg")

# Option B: From local file
# img = load_image("images/my_photo.jpg")

# Option C: Search and browse
# results = search_images("angel statue", count=8)
# for i, r in enumerate(results):
#     print(f"  [{i}] {r['description'][:60]} — {r['photographer']}")
# img = load_image(results[0]["url"])  # pick one

SUBJECT = "cat"  # label for saving later

show_image(img, title=f"Source image: {SUBJECT} ({img.size[0]}×{img.size[1]})")

In [ ]:
# Cell 3: Edge detection — tune Canny thresholds

# ---- EDIT THESE to experiment ----
CANNY_LOW = 50
CANNY_HIGH = 150
BLUR_KERNEL = 5
# ----------------------------------

edges = detect_edges(img, method="canny", low=CANNY_LOW, high=CANNY_HIGH, blur_kernel=BLUR_KERNEL)

show_side_by_side(img, edges, "Original", f"Canny Edges (low={CANNY_LOW}, high={CANNY_HIGH})")

In [ ]:
# Cell 4: SVG tracing — convert edge map to vector paths

# ---- vtracer parameters ----
FILTER_SPECKLE = 4        # Remove small noise (higher = fewer small strokes)
CORNER_THRESHOLD = 60     # Angle threshold for corners (degrees)
LENGTH_THRESHOLD = 4.0    # Minimum path length
SPLICE_THRESHOLD = 45     # Angle threshold for splicing paths
PATH_PRECISION = 3        # Decimal precision for SVG coordinates
# ----------------------------

t0 = time.time()
svg = trace_to_svg(
    edges,
    filter_speckle=FILTER_SPECKLE,
    corner_threshold=CORNER_THRESHOLD,
    length_threshold=LENGTH_THRESHOLD,
    splice_threshold=SPLICE_THRESHOLD,
    path_precision=PATH_PRECISION,
)
svg_time = time.time() - t0

# Count paths in SVG
import re
path_count = len(re.findall(r'd="', svg))
print(f"SVG tracing: {path_count} paths in {svg_time:.3f}s")
print(f"SVG size: {len(svg):,} chars")

In [ ]:
# Cell 5: Stroke extraction — parse SVG → composition

# ---- Simplification ----
SIMPLIFY_TOLERANCE = 0.005  # Douglas-Peucker tolerance (higher = fewer points, sketchier)
# ------------------------

t0 = time.time()
strokes = svg_to_strokes(svg, simplify_tolerance=SIMPLIFY_TOLERANCE)
stroke_time = time.time() - t0

comp = Composition(
    width=255, height=255,
    doodle_fragments=[__import__('helpers.models', fromlist=['DoodleFragment']).DoodleFragment(strokes=strokes)],
    tags=["traced", "traced-canny", SUBJECT],
)

is_valid, score = validate(comp)
print(f"Extracted {len(strokes)} strokes, {count_points(comp)} total points in {stroke_time:.3f}s")
print(f"Valid: {is_valid}, Quality: {score:.4f}")

fig = draw_grid([comp], cols=1, title=f"Traced '{SUBJECT}' (q={score:.3f}, s={len(strokes)}, p={count_points(comp)})", figsize_per_cell=5)
plt.show()

In [ ]:
# Cell 6: Full pipeline — one-liner convenience

t0 = time.time()
traced = trace_image(
    img,
    method="canny", low=CANNY_LOW, high=CANNY_HIGH,
    simplify_tolerance=SIMPLIFY_TOLERANCE,
    filter_speckle=FILTER_SPECKLE,
    subject=SUBJECT,
)
total_time = time.time() - t0

is_valid, score = validate(traced)
print(f"Full pipeline: {total_time:.3f}s")
print(f"Strokes: {count_strokes(traced)}, Points: {count_points(traced)}, Quality: {score:.4f}")

# Show original image alongside traced composition
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
import numpy as np
ax1.imshow(np.array(img))
ax1.set_title("Original")
ax1.set_xticks([]); ax1.set_yticks([])
draw(traced, ax=ax2, title=f"Traced (q={score:.3f})")
fig.tight_layout()
plt.show()

In [ ]:
# Cell 7: Parameter tuning — sweep Canny thresholds

param_sets = [
    {"low": 30, "high": 100, "simplify_tolerance": 0.003},
    {"low": 50, "high": 150, "simplify_tolerance": 0.005},
    {"low": 80, "high": 200, "simplify_tolerance": 0.005},
    {"low": 100, "high": 250, "simplify_tolerance": 0.008},
    {"low": 50, "high": 150, "simplify_tolerance": 0.015},
]

traced_variants = trace_with_params(img, param_sets, subject=SUBJECT)

titles = []
for i, (comp, params) in enumerate(zip(traced_variants, param_sets)):
    is_valid, score = validate(comp)
    s = count_strokes(comp)
    p = count_points(comp)
    label = f"lo={params.get('low', 50)} hi={params.get('high', 150)}\nsimp={params.get('simplify_tolerance', 0.005)}\nq={score:.3f} s={s} p={p}"
    titles.append(label)
    print(f"  [{i}] low={params.get('low')}, high={params.get('high')}, simp={params.get('simplify_tolerance')}: q={score:.4f}, s={s}, p={p}")

# Draw all variants in a grid
n = len(traced_variants)
fig, axes = plt.subplots(1, n, figsize=(4 * n, 4))
if n == 1:
    axes = [axes]
for i, (comp, title) in enumerate(zip(traced_variants, titles)):
    draw(comp, ax=axes[i], title=title)
fig.suptitle(f"Parameter Sweep: '{SUBJECT}'", fontsize=14, fontweight="bold")
fig.tight_layout()
plt.show()

In [ ]:
# Cell 8: Edge method comparison

methods = [
    {"method": "canny", "low": 50, "high": 150},
    {"method": "canny", "low": 30, "high": 100},
    {"method": "adaptive"},
]

method_results = trace_with_params(img, methods, subject=SUBJECT)

fig, axes = plt.subplots(1, len(methods) + 1, figsize=(4 * (len(methods) + 1), 4))
import numpy as np
axes[0].imshow(np.array(img))
axes[0].set_title("Original")
axes[0].set_xticks([]); axes[0].set_yticks([])

for i, (comp, params) in enumerate(zip(method_results, methods)):
    is_valid, score = validate(comp)
    method = params.get('method', 'canny')
    draw(comp, ax=axes[i + 1], title=f"{method}\nq={score:.3f} s={count_strokes(comp)}")

fig.suptitle(f"Edge Method Comparison: '{SUBJECT}'", fontsize=14, fontweight="bold")
fig.tight_layout()
plt.show()

In [ ]:
# Cell 9: Vision labeling (optional) — auto-detect subject and tags
# Requires a vision model in Ollama (e.g., qwen2.5-vl:7b) or Claude API key

# Option A: Ollama vision model (local, free)
try:
    print("Labeling with Ollama vision model...")
    labels = label_with_ollama(img)
    print(f"  Subject: {labels['subject']}")
    print(f"  Tags: {labels['tags']}")
    print(f"  Description: {labels['description']}")
    SUBJECT = labels["subject"]  # Update subject for saving
except Exception as e:
    print(f"  Ollama vision not available: {e}")
    print("  To use: ollama pull qwen2.5-vl:7b")

# Option B: Claude (cloud, costs money)
# from helpers.claude import UsageTracker
# tracker = UsageTracker()
# labels = label_with_claude(img, tracker=tracker)
# print(f"Subject: {labels['subject']}, Tags: {labels['tags']}")
# print(f"Cost: {tracker.summary()}")

In [ ]:
# Cell 10: Compare traced vs curated Quick Draw data
try:
    curated = get_curated(SUBJECT, limit=10)
    if curated:
        print(f"Found {len(curated)} curated '{SUBJECT}' compositions")
        fig = draw_comparison(
            curated[:5], [traced],
            cols=5, title=f"'{SUBJECT}': Curated Quick Draw (top) vs Traced Photo (bottom)"
        )
        plt.show()
    else:
        print(f"No curated data for '{SUBJECT}'")
        fig = draw_grid([traced], cols=1, title=f"Traced '{SUBJECT}'", figsize_per_cell=5)
        plt.show()
except Exception as e:
    print(f"DB not available: {e}")
    fig = draw_grid([traced], cols=1, title=f"Traced '{SUBJECT}'", figsize_per_cell=5)
    plt.show()

In [ ]:
# Cell 11: Batch tracing — multiple images

urls = [
    "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg",
    # Add more URLs here:
    # "https://example.com/another-image.jpg",
]

batch_results = []
for i, url in enumerate(urls):
    try:
        batch_img = load_image(url)
        batch_comp = trace_image(batch_img, low=CANNY_LOW, high=CANNY_HIGH, simplify_tolerance=SIMPLIFY_TOLERANCE, subject=SUBJECT)
        is_valid, score = validate(batch_comp)
        batch_results.append(batch_comp)
        print(f"  [{i}] {url[:60]}... → q={score:.4f}, s={count_strokes(batch_comp)}, p={count_points(batch_comp)}")
    except Exception as e:
        print(f"  [{i}] {url[:60]}... → ERROR: {e}")

if batch_results:
    fig = draw_grid(batch_results, cols=min(len(batch_results), 5), title=f"Batch Traced: '{SUBJECT}'")
    plt.show()

In [ ]:
# Cell 12: Save to database
# Only run when you're happy with the results!

to_save = [traced] + [c for c in batch_results if validate(c)[0]] if batch_results else [traced]
to_save = [c for c in to_save if validate(c)[0]]
print(f"{len(to_save)} valid compositions ready to save")

for i, comp in enumerate(to_save):
    is_valid, score = validate(comp)
    print(f"  [{i}] q={score:.4f}, s={count_strokes(comp)}, p={count_points(comp)}")

# Uncomment to save:
# saved = save_compositions(SUBJECT, to_save, generation_method="traced-canny")
# print(f"Saved {saved} compositions to database")

## Tips for Tuning Trace Parameters

### Canny Edge Detection
- **low/high thresholds**: Lower values = more edges (more detail, more noise). Higher = cleaner but may lose features.
- **Typical ranges**: low=30-100, high=100-250. Try `low=50, high=150` as a starting point.
- **blur_kernel**: Larger values smooth the image more before edge detection. Use 3-7.

### vtracer Parameters
- **filter_speckle**: Higher values remove small noise strokes (default 4, try 2-10).
- **corner_threshold**: Controls how sharp corners must be to be kept (degrees, default 60).
- **length_threshold**: Minimum path length to keep (default 4.0).

### Simplification
- **simplify_tolerance**: Douglas-Peucker tolerance. Higher = fewer points, sketchier look.
  - 0.002 — very detailed, many points
  - 0.005 — balanced (default)
  - 0.010 — simplified, fewer points
  - 0.020 — very sketchy, minimal points

### Image Source Tips
- **High contrast** photos trace better than low-contrast ones.
- **Simple backgrounds** (white, solid color) produce cleaner edge maps.
- **Subject should fill the frame** — the composition is normalized to fill the canvas.
- Use `search_images("cat white background")` for clean source images.